In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

In [2]:
try:
    conexionpeliculas=psycopg2.connect(
    database="film",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [3]:
# Primero, deshacer la transacción fallida
conexionpeliculas.rollback()

cursorpeliculas = conexionpeliculas.cursor()
cursorpeliculas

<cursor object at 0x0000026A608B04A0; closed: 0>

In [4]:
# Tabla peliculas

# Tipo       object
# Titulo     object
# Año         int64
# Mes         int64
# Id         object
# generos    object

querydecreacion1 = """
    CREATE TABLE IF NOT EXISTS peliculas (
    tipo VARCHAR,
    titulo VARCHAR,
    año INT,
    mes INT,
    id_pelicula VARCHAR PRIMARY KEY,
    genero VARCHAR
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion1)
conexionpeliculas.commit()

# Tabla detalles

# titulo          object
# cali_imdb      float64
# direc_imbd      object
# guion_imbd      object
# argum_imbd      object
# durac_imbd      object
# id_pelicula     object

querydecreacion2 = """
    CREATE TABLE IF NOT EXISTS detalles_pelicula(
    titulo VARCHAR,
    cali_imdb DECIMAL,
    direc_imbd VARCHAR,
    guion_imbd VARCHAR,
    argum_imbd VARCHAR,
    durac_imbd VARCHAR,
    id_pelicula VARCHAR PRIMARY KEY,
    FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion2)
conexionpeliculas.commit()

# Tabla detalles actores

# actr            object
# año_nac          int64
# conocido_por    object
# que_hace        object
# premios         object

querydecreacion4 = """
    CREATE TABLE IF NOT EXISTS detalles_actores (
    nombre VARCHAR PRIMARY KEY,
    año INT,
    conocido VARCHAR,
    rol VARCHAR,
    premios VARCHAR
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion4)
conexionpeliculas.commit()

# Tabla actores

# Movie ID        object
# Actriz_Actor    object

querydecreacion3 = """
    CREATE TABLE IF NOT EXISTS actores (
    id_pelicula VARCHAR,
    nombre VARCHAR,
    FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula),
    FOREIGN KEY (nombre) REFERENCES detalles_actores (nombre)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion3)
conexionpeliculas.commit()



In [5]:
df_peliculas=pd.read_csv("../data/extra/peliculas.csv")
df_peliculas.sample(5)

,Tipo,Titulo,Año,Mes,Id,generos
574,Short,Hurst,2011,6,tt1999895,Horror
865,Short,Ghunghroo,2023,5,tt27812992,Horror
721,Short,Into the Black,2017,0,tt5866170,Horror
1128,Movie,Savageland,2020,5,tt3899154,Thriller
263,Movie,Because We Are,2021,8,tt10692368,Drama


In [6]:
df_peliculas.dtypes

Tipo       object
Titulo     object
Año         int64
Mes         int64
Id         object
generos    object
dtype: object

In [7]:
lista_peliculas = [tuple(row) for row in df_peliculas.values]

In [8]:
df_detalles_peliculas=pd.read_csv("../data/extra/detalles_peliculas.csv")
df_detalles_peliculas.sample(5)

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
1010,OftalmÃ³s,8.5,Pedro Jaén R.,Pedro Jaén R.,Desconocido,16min,tt1155067
1208,12 Feet Deep,5.2,Desconocido,Desconocido,Desconocido,1h 25min,tt5143226
1366,Jacquerie,NaN,Janos Killius,Janos Killius,Desconocido,6min,tt4360490
412,Life As We Know It,NaN,Desconocido,Desconocido,Desconocido,Desconocido,tt10021716
818,Toby Doll,NaN,Sylvain Pelissier,Sylvain Pelissier,Desconocido,Desconocido,tt12245110


In [9]:
df_detalles_peliculas.dtypes

titulo          object
cali_imdb      float64
direc_imbd      object
guion_imbd      object
argum_imbd      object
durac_imbd      object
id_pelicula     object
dtype: object

In [10]:
lista_detalles_peliculas = [tuple(row) for row in df_detalles_peliculas.values]

In [11]:
df_detalles_actores=pd.read_csv("../data/extra/detalles_artistas.csv")
df_detalles_actores.sample(5)

,actr,año_nac,conocido_por,que_hace,premios
8481,Sharn Hammond,9999,"'Paula's Guide to Love', 'Lost and Found', 'De...","Actor, Writer, Producer",no tiene premios
4566,Johanna ter Steege,1961,"'Desaparecida', 'Amor inmortal', 'Tot ziens', ...",Actress,1 win & 1 nomination
9015,Tammy Lobb,9999,'Backhand',Actress,no tiene premios
6186,Masiela Lusha,1985,"'El mal que esconde el olvido', 'El último Sha...","Actress, Stunts, Camera and Electrical Department",no tiene premios
7499,Predrag 'Miki' Manojlovic,1950,"'Underground', 'The Heir Apparent: Largo Winch...",Actress,no tiene premios


In [12]:
df_detalles_actores.dtypes

actr            object
año_nac          int64
conocido_por    object
que_hace        object
premios         object
dtype: object

In [13]:
df_detalles_actores.drop_duplicates(inplace=True)

In [14]:
df_detalles_actores.shape

(9907, 5)

In [15]:
df_detalles_actores["Actriz_Actor"]=df_detalles_actores["actr"]

In [16]:
df_detalles_actores.nunique()

actr            9907
año_nac          115
conocido_por    9301
que_hace        1076
premios          872
Actriz_Actor    9907
dtype: int64

In [18]:
df_actores=pd.read_csv("../data/extra/pelicula_artista.csv")
df_actores.sample(5)

,Movie ID,Actriz_Actor
4453,tt0469754,Benedikt Erlingsson
13165,tt0425333,Celal Belgil
7220,tt0385212,AiAi Delas Alas
1336,tt12237116,Luna Wolf
11076,tt1871270,Hennely Jimenez


In [19]:
df_actores["Actriz_Actor"].value_counts()

Actriz_Actor
Sin datos         4094
Shakti Kapoor        8
Anupam Kher          8
Gulshan Grover       6
Mohnish Behl         6
                  ... 
Keep Chambers        1
Jarod Johnson        1
Joe D'Aguanno        1
Darren Baxter        1
Fanta Berete         1
Name: count, Length: 9909, dtype: int64

In [20]:
df_actores.dtypes

Movie ID        object
Actriz_Actor    object
dtype: object

In [21]:
df_actores.shape

(14529, 2)

In [22]:
df_actores.drop_duplicates(inplace=True)

In [23]:
df_actores=pd.merge(df_actores,df_detalles_actores, on="Actriz_Actor",how="inner")

In [24]:
df_actores

,Movie ID,Actriz_Actor,actr,año_nac,conocido_por,que_hace,premios
0,tt0110660,Tony Ka Fai Leung,Tony Ka Fai Leung,1958,"'Election', 'The Lover', 'Cold War', 'Men Sudd...","Actor, Producer, Director",16 wins & 42 nominations
1,tt0110363,Luke Edwards,Luke Edwards,1980,"'Little Big League', 'The Wizard', 'Jeepers Cr...","Actor, Director, Writer",no tiene premios
2,tt0110397,Jennifer Barker,Jennifer Barker,9999,"'Traffic', 'Born Wild', 'Murder One', 'Centuri...",Actor,no tiene premios
3,tt0111045,Miklós Déri,Miklós Déri,9999,"'Knights of the Zodiac', 'Shadow and Bone', 'H...","Actor, Writer, Producer",3 nominations
4,tt0111653,Joe Bays,Joe Bays,1955,"'Spider-Man 3', 'Office Space', 'Space Jam', '...",Actor,no tiene premios
...,...,...,...,...,...,...,...
11224,tt10709060,Kendra Kuhnhofer,Kendra Kuhnhofer,9999,'Game Over',"Camera and Electrical Department, Cinematograp...",no tiene premios
11225,tt14142640,Prakash,Prakash,1965,"'Anniyan', 'Singham', 'Abhiyum Naanum', 'Kalki'","Actor, Stunts, Director",1 win & 6 nominations
11226,tt18688348,Oleg Primogenov,Oleg Primogenov,1955,"'ToyKhtoProyshovKrizVohon', 'Toloka', 'Ballada...",Actor,1 nomination
11227,tt27241513,Georgiy Topolaga,Georgiy Topolaga,1965,"'The Blackout', 'Khitrovka. The Sign of Four',...",Actress,no tiene premios


In [25]:
df_actores.drop(columns=["actr","año_nac","conocido_por","que_hace","premios"], inplace=True)

In [26]:
lista_actores = [tuple(row) for row in df_actores.values]

In [34]:
df_detalles_actores=df_detalles_actores.drop(columns="Actriz_Actor")

In [35]:
lista_detalles_actores = [tuple(row) for row in df_detalles_actores.values]

In [32]:
# Primero, deshacer la transacción fallida
conexionpeliculas.rollback()

cursorpeliculas = conexionpeliculas.cursor()
cursorpeliculas

<cursor object at 0x0000026A62C704A0; closed: 0>

In [28]:
queryinsercion = "INSERT INTO peliculas (tipo, titulo, año, mes, id_pelicula, genero) VALUES (%s,%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_peliculas
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

In [29]:
queryinsercion = "INSERT INTO detalles_pelicula (titulo, cali_imdb, direc_imbd, guion_imbd, argum_imbd, durac_imbd, id_pelicula) VALUES (%s,%s,%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_detalles_peliculas
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

In [36]:
queryinsercion = "INSERT INTO detalles_actores (nombre, año, conocido, rol, premios) VALUES (%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_detalles_actores
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

In [37]:
queryinsercion = "INSERT INTO actores (id_pelicula, nombre) VALUES (%s,%s);"
listavaloresinsercion=lista_actores
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()